# Building a Neural Net using AutoDiff
In this notebook, we demostrate the use of our AutoDiff package. We implement a Neural Network, with a single hidden layer, with the Sigmoid activation function. We implement forward and backward passes for the Network, and train it using Stochastic Gradient descent with a batch size of 1. All gradient computations required for training this Neural Net have been implemented using our AutoDiff package. Specifically, we create a computational graph before training wherein each Node of the Neural Net is an instance of our AutoDiff's class `DiffObj`. During training:
* For a forward pass, we invoke the `DiffObj` method `get_val` to compute values of various nodes.
* For backward pass, we invoke the `DiffObj` method `get_der` to compute various gradients. Mostly, we only need gradients w.r.t Network weights, so we use the `with_respect_to` argument of the `get_der` method to compute partial gradients.

For demonstration, we train a Neural Net (with 10 nodes in the hidden layer) on synthetic data generated using the function $f(x) = sin(x^2)$. We then compute the predictions on our Test Set (which is also generated using $f(x)$), and plot the truth value and prediction.

In [6]:
from AutoDiff import DiffObj, Variable, Constant
from AutoDiff import MathOps as mo
from AutoDiff.nn import NeuralNet
import numpy as np
import matplotlib.pyplot as plt

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [7]:
import numpy as np

In [8]:
# Demonstration of Neural Network
input_dim = 1
hidden_dim = 12
out_dim = 1
lr = 0.1
nn = NeuralNet(input_dim, hidden_dim, out_dim, lr)

X_data = np.random.randn(3000, input_dim)
y_data = np.sin(np.multiply(X_data, X_data))
num_train = 2700
X_train, y_train = X_data[0:num_train,:], y_data[0:num_train]
X_test, y_test = X_data[num_train:,:], y_data[num_train:]

In [ ]:
num_epochs = 60
tol = 1e-4
prev_loss = 1e6
for epoch in range(num_epochs):
    running_loss = 0
    for i in range(X_train.shape[0]):
        loss, _ = nn.forward(X_train[i,:], y_train[i])
        running_loss += loss[0]
        nn.backward()
    print('Loss after epoch {} is: {:.4f}'.format(epoch + 1, running_loss))
    if abs(running_loss - prev_loss) < tol:
        print('Training converged, stopping early.')
        break
    prev_loss = running_loss

In [ ]:
# Test on the test set
y_pred = [0.0]*len(y_test)
for i in range(len(y_test)):
    _, pred = nn.forward(X_test[i,:], y_test[i])
    y_pred[i] = list(pred)[0]

In [ ]:
fig = plt.gcf()
fig.set_size_inches(9, 6)
_ = plt.scatter(X_test[:,0], y_test, marker='o', s=10, label=r'Ground Truth ($f(x) = sin(x^2)$)')
_ = plt.scatter(X_test[:,0], y_pred, marker='v', s=10, label='Prediction (on Test Set)')
_ = plt.xlabel('X', fontsize=14)
_ = plt.ylabel('y', fontsize=14)
_ = plt.legend(loc='best', fontsize=14)
_ = plt.title('Ground Truth vs Predictions (after {} epochs)'.format(num_epochs), fontsize=14)
fig.savefig('neural_net.png')